In [2]:
with open('data/Test_second/epigraphic_corpus-2.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [4]:
lines[0]

'<doc id="file32708451" filename="Epigrafika" parent_folder="web_wikipedia" url="https://uk.wikipedia.org/wiki/%D0%95%D0%BF%D1%96%D0%B3%D1%80%D0%B0%D1%84%D1%96%D0%BA%D0%B0">\n'

In [14]:
def split_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    total_lines = len(lines)
    
    lines_per_part = total_lines // 40

    remainder = total_lines % 40
    
    output_dir = "data/Test_second"
    
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    start = 0
    for i in range(40):
        if i < remainder:
            lines_in_part = lines_per_part + 1
        else:
            lines_in_part = lines_per_part
        
        end = start + lines_in_part
        part_lines = lines[start:end]

        part_file_path = os.path.join(output_dir, f"part_{i + 1}.txt")
        with open(part_file_path, 'w', encoding='utf-8') as part_file:
            part_file.writelines(part_lines)

        start = end
    
    print(f"Original file has been seperated to 40 parts, and saved in {output_dir}.")

# 使用示例
split_txt_file('data/Test_second/epigraphic_corpus-2.txt')


Original file has been seperated to 20 parts, and saved in data/Test_second.


In [21]:
import spacy
model_path = 'model/first_model'
nlp = spacy.load(model_path)

In [22]:
from tqdm import tqdm
import spacy
import pandas as pd

# Load the spaCy model
nlp = spacy.load('uk_core_news_lg')

# Assume 'annotations' DataFrame is loaded and combined
# Tokenize keywords in advance
annotation_path = 'data/annotation/annotation.csv'
annotations = pd.read_csv(annotation_path)
annotations['keyword_tokens'] = annotations['keywords'].apply(lambda kw: [token.text for token in nlp(kw)])

# List of file paths
files = [
    "data/Test_second/part_1.txt",
    "data/Test_second/part_2.txt",
    "data/Test_second/part_3.txt",
    "data/Test_second/part_4.txt",
    "data/Test_second/part_5.txt",
    "data/Test_second/part_6.txt",
    "data/Test_second/part_7.txt",
    "data/Test_second/part_8.txt",
    "data/Test_second/part_9.txt",
    "data/Test_second/part_10.txt",
    "data/Test_second/part_11.txt",
    "data/Test_second/part_12.txt",
    "data/Test_second/part_13.txt",
    "data/Test_second/part_14.txt",
    "data/Test_second/part_15.txt",
    "data/Test_second/part_16.txt",
    "data/Test_second/part_17.txt",
    "data/Test_second/part_18.txt",
    "data/Test_second/part_19.txt",
    "data/Test_second/part_20.txt",
    "data/Test_second/part_21.txt",
    "data/Test_second/part_22.txt",
    "data/Test_second/part_23.txt",
    "data/Test_second/part_24.txt",
    "data/Test_second/part_25.txt",
    "data/Test_second/part_26.txt",
    "data/Test_second/part_27.txt",
    "data/Test_second/part_28.txt",
    "data/Test_second/part_29.txt",
    "data/Test_second/part_30.txt",
    "data/Test_second/part_31.txt",
    "data/Test_second/part_32.txt",
    "data/Test_second/part_33.txt",
    "data/Test_second/part_34.txt",
    "data/Test_second/part_35.txt",
    "data/Test_second/part_36.txt",
    "data/Test_second/part_37.txt",
    "data/Test_second/part_38.txt",
    "data/Test_second/part_39.txt",
    "data/Test_second/part_40.txt",
    # Assume other files as needed
]

# Prepare to write to the TSV file
with open("data/Test_second/data/data.tsv", "w", encoding='utf-8') as f:
    # Process each file
    for file_path in tqdm(files, total=len(files)):
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        doc = nlp(text.strip())

        # Iterate over the sentences
        for sent in doc.sents:
            sent_tokens = [token.text for token in sent if token.text not in ['\t', '\n', ' ']]
            token_labels = ['O' for _ in sent_tokens]  # Initialize labels as 'O' for each token

            # Check for matching keyword token sequences
            for _, row in annotations.iterrows():
                keyword_tokens = row['keyword_tokens']
                for i in range(len(sent_tokens) - len(keyword_tokens) + 1):
                    if sent_tokens[i:i + len(keyword_tokens)] == keyword_tokens:
                        # Match found, apply BIO tagging
                        if all(label == 'O' for label in token_labels[i:i+len(keyword_tokens)]):  # Ensure not to overwrite existing tags
                            token_labels[i] = f'B-{row["type"]}'
                            for j in range(1, len(keyword_tokens)):
                                token_labels[i + j] = f'I-{row["type"]}'

            # Write tokens and tags to file, with newline after each sentence
            for token, label in zip(sent_tokens, token_labels):
                f.write(f"{token}\t{label}\n")
            f.write("\n")  # Newline after each sentence to separate them


  0%|                                                                                           | 0/40 [00:18<?, ?it/s]


KeyboardInterrupt: 

In [23]:
def load_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.read().strip().split('\n\n')  # Splitting data by double newlines

    formatted_data = []
    for record in data:
        text, entities = '', []
        lines = record.split('\n')
        start = 0  # Initialize start of the first token
        entity_start = None  # Initialize the start of an entity
        entity_type = None  # Initialize the entity type
        for line in lines:
            parts = line.split('\t')
            if len(parts) >= 2:  # Ensure there's at least a token and a tag
                token, tag = parts[0], parts[1]
                # Update text with token followed by space
                text += token + ' '
                end = start + len(token)  # End index of the current token
                if tag.startswith('B-') or tag.startswith('I-'):  # Check if the tag indicates an entity
                    if tag.startswith('B-'):
                        # If there was a previous entity, append it
                        if entity_start is not None:
                            entities.append((entity_start, prev_end, entity_type))
                        # Start new entity
                        entity_start = start
                        entity_type = tag.split('-')[1]
                    if tag.startswith('I-') and entity_start is not None:
                        # Continue the entity
                        entity_type = tag.split('-')[1]  # Update the type if needed (typically not necessary)
                else:
                    # If ending part of entity, append it
                    if entity_start is not None:
                        entities.append((entity_start, end, entity_type))
                        entity_start = None  # Reset for the next entity

                # Prepare for next token
                start = end + 1
                prev_end = end  # Remember the end of the last token

        # Check for any trailing entity at the end
        if entity_start is not None:
            entities.append((entity_start, prev_end, entity_type))

        # Append the parsed data if there are entities
        if entities:
            formatted_data.append((text.strip(), {'entities': entities}))

    return formatted_data

# Load the data
second_data = load_data('data/Test_second/data/data.tsv')

In [24]:
second_data

[("Історики виділяють декілька основних видів епіграфічних пам'яток , які виконують різну функцію : < /p > < p > Вотивні написи — в них висловлюється подяка певному божеству ( « Коли божество X вчинить дію XY для особи Y , то особа Y подякує божеству , зробивши напис » ) .",
  {'entities': [(259, 266, 'inscription')]}),
 ('< /p > < p > Гоноративні написи — в них прославляються вчинки певної людини , зазвичай є також короткий опис цих вчинків .',
  {'entities': [(104, 112, 'inscription')]}),
 ('Переважно це графіті — написи , нанесені гострим знаряддям , як правило , на керамічному посуді та його уламках .',
  {'entities': [(13, 22, 'inscription')]}),
 ('З VI — V століття до н . е . використовувалися для написів черепки як найдешевший матеріал .',
  {'entities': [(82, 92, 'material')]}),
 ('Часом траплялися графіті на свинцевих пластинах з листами і заклинаннями .',
  {'entities': [(17, 27, 'inscription')]}),
 ('До сьогодні збереглися деякі давньоруські написи дохристиянського часу : гр

In [34]:
second_data[0][1]['entities'][0][2]

'inscription'

In [45]:
for each_data in second_data:
    print(each_data[0])
    break

Історики виділяють декілька основних видів епіграфічних пам'яток , які виконують різну функцію : < /p > < p > Вотивні написи — в них висловлюється подяка певному божеству ( « Коли божество X вчинить дію XY для особи Y , то особа Y подякує божеству , зробивши напис » ) .


In [46]:
num_of_sentence = len(second_data)
token_count = 0
num_entities_all = 0
num_of_each_entity_type = {}
for each_data in second_data:
    entities = each_data[1]['entities']
    sentence = each_data[0]
    doc = nlp(sentence)
    token_count += len(doc)
    num_entities_all += len(entities)
    for entity in entities:
        entity_type = entity[2]
        if entity_type in num_of_each_entity_type:
            num_of_each_entity_type[entity_type] += 1
        else:
            num_of_each_entity_type[entity_type] = 1

In [47]:
num_of_each_entity_type

{'inscription': 14970,
 'material': 483,
 'other': 1354,
 'dating_criteria': 4268,
 'object_type': 698,
 'monument': 355,
 'monument_subtype': 12,
 'epigraphic_shorthand': 33,
 'decoration': 961,
 'preservation_state': 150,
 'inscription_type': 1462,
 'monument_type': 1,
 'execution_technique': 799,
 'symbol': 31,
 'inscripiton type': 1}

In [48]:
del num_of_each_entity_type['inscripiton type']
num_of_each_entity_type['inscription_type'] += 1

In [49]:
num_of_each_entity_type

{'inscription': 14970,
 'material': 483,
 'other': 1354,
 'dating_criteria': 4268,
 'object_type': 698,
 'monument': 355,
 'monument_subtype': 12,
 'epigraphic_shorthand': 33,
 'decoration': 961,
 'preservation_state': 150,
 'inscription_type': 1463,
 'monument_type': 1,
 'execution_technique': 799,
 'symbol': 31}

In [50]:
num_entities_all

25578

In [51]:
num_of_sentence

20093

In [52]:
token_count

483836

In [53]:
statistics_dict = {"number_of_sentence":num_of_sentence, "number_of_tokens":token_count, "number_of_entities_all":num_entities_all}

In [58]:
for each_type in num_of_each_entity_type:
    key = "number_of_" + each_type
    statistics_dict[key] = num_of_each_entity_type[each_type]

In [68]:
import pandas as pd
keys = list(statistics_dict.keys())
values = list(statistics_dict.values())
statistics_df = pd.DataFrame({"name":keys, "number":values})

In [70]:
statistics_df

,name,number
0,number_of_sentence,20093
1,number_of_tokens,483836
2,number_of_entities_all,25578
3,number_of_inscription,14970
4,number_of_material,483
5,number_of_other,1354
6,number_of_dating_criteria,4268
7,number_of_object_type,698
8,number_of_monument,355
9,number_of_monument_subtype,12


In [71]:
statistics_df.to_csv("data/Test_second/data/statistics.csv")